In [1]:
import ipynb.fs.full.preprocess_eda as preprocess

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\agarw\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\agarw\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\agarw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


The length of the training data:  7500
The length of the testing data:  4500
The length of the validation data:  3000


In [2]:
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import GlobalMaxPool1D
from tensorflow.keras.layers import Flatten

In [3]:
import keras_tuner

In [4]:
train_obj = preprocess.pre_process(preprocess.train_df)
val_obj = preprocess.pre_process(preprocess.val_df)
test_obj = preprocess.pre_process(preprocess.test_df)

In [5]:
x_train = train_obj.preprocess()
x_train = train_obj.lemmatise()

x_val = val_obj.preprocess()
x_val = val_obj.lemmatise()

x_test = test_obj.preprocess()
x_test = test_obj.lemmatise()

In [6]:
y_train = train_obj.encode_class()
y_val = val_obj.encode_class()
y_test = test_obj.encode_class()

In [7]:
# vocab = list(set(" ".join(preprocess.pd.concat([x_train, x_val, x_test])).split()))

In [8]:
vocab_size = 20000
sequence_length = 25

In [40]:
textvector_layer = TextVectorization(max_tokens=vocab_size, 
                                     ngrams=(1, 2, 3),
                                     output_mode="tf_idf", 
                                     pad_to_max_tokens=True)
                                    # output_sequence_length=25)

In [41]:
textvector_layer.adapt(preprocess.pd.concat([x_train, x_val, x_test],
                                            ignore_index=True))

In [11]:
# textvector_layer.get_vocabulary()

In [12]:
# print(textvector_layer(x_train).numpy()[0])

In [13]:
hp = keras_tuner.HyperParameters()


In [14]:
class lstm():
    def lstm_model(self, dropout, dropout_rate, convolution, convolution2, 
                   filter, kernel_size, activation, pool_size, units):
        model = Sequential()
        model.add(textvector_layer)
        model.add(Embedding(input_dim=vocab_size+1, 
                            output_dim=sequence_length))
        
        if dropout:
            model.add(Dropout(rate=dropout_rate))
        else:
            model.add(SpatialDropout1D(rate=dropout_rate))
        
        if convolution:
            model.add(Conv1D(filters=filter, 
                             kernel_size=kernel_size, 
                             padding="same",
                             activation=activation))
            model.add(MaxPooling1D(pool_size=pool_size))
        
        if convolution2:
            model.add(Conv1D(filters=filter, 
                             kernel_size=kernel_size, 
                             padding="same",
                             activation=activation))
            model.add(MaxPooling1D(pool_size=pool_size))
        
        model.add(LSTM(units=units, 
                       dropout=dropout_rate,
                       recurrent_dropout=dropout_rate, 
                       activation=activation))
        

In [19]:

def build_model(hp):
    model = Sequential()
    model.add(textvector_layer)
    model.add(Embedding(input_dim=vocab_size+1, 
                        output_dim=sequence_length))
    
    # dropout layer 1
    dropout_rate1 = hp.Float("dropout_rate1", 
                             min_value=0.2, 
                             max_value=0.8, 
                             step=0.025)
    if hp.Boolean("dropout1"):
            model.add(Dropout(rate=dropout_rate1))
    
    # convolutional layer 1
    if hp.Boolean("convolution1"):
        model.add(Conv1D(filters=hp.Int("filter1", 
                                            min_value=32, 
                                            max_value=512, 
                                            step=32), 
                            kernel_size=hp.Int("kernel_size1", 
                                            min_value=2, 
                                            max_value=8, 
                                            step=1), 
                            padding="same",
                            activation="relu"))
        if hp.Boolean("pooling1"):
            model.add(MaxPooling1D(pool_size=hp.Int("pool_size1", 
                                                    min_value=2, 
                                                    max_value=8, 
                                                    step=1), 
                                                    padding="same"))
        else:
            model.add(GlobalMaxPool1D())
    
    # dropout layer 2
    dropout_rate2 = hp.Float("dropout_rate1", 
                             min_value=0.2, 
                             max_value=0.8, 
                             step=0.025)
    if hp.Boolean("dropout2"):
         model.add(Dropout(rate=dropout_rate2))
    
    # convolutional layer 2
    if hp.Boolean("convolution2"):
        model.add(Conv1D(filters=hp.Int("filter2", 
                                            min_value=32, 
                                            max_value=512, 
                                            step=32), 
                            kernel_size=hp.Int("kernel_size2", 
                                            min_value=2, 
                                            max_value=8, 
                                            step=1), 
                            padding="same",
                            activation="relu"))
        if hp.Boolean("pooling2"):
            model.add(MaxPooling1D(pool_size=hp.Int("pool_size2", 
                                                    min_value=2, 
                                                    max_value=8, 
                                                    step=1), padding="same"))
        else:
            model.add(GlobalMaxPool1D())

    if hp.Boolean("dropout3"):
            model.add(Dropout(rate=hp.Float("dropout_rate3", 
                                            min_value=0.2, 
                                            max_value=0.8, 
                                            step=0.025)))

    
    model.add(LSTM(units=hp.Int("lstm_units",
                                min_value=32, 
                                max_value=1024, 
                                step=32)))
    
    if hp.Boolean("dropout4"):
            model.add(Dropout(rate=hp.Float("dropout_rate4", 
                                            min_value=0.2, 
                                            max_value=0.8, 
                                            step=0.025)))
        
    model.add(Flatten())
    model.add(Dense(units=150,
                    activation="softmax"))
    lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    
    # model.compile(optimizer=hp.Choice("optimizer", 
    #                                   ["keras.optimizers.Adam(learning_rate=lr)",
    #                                    "keras.optimizers.Nadam(learning_rate=lr)",
    #                                    "keras.optimizers.SGD(learning_rate=lr)"]), 
    #               loss=hp.Choice("loss", ["keras.losses.CategoricalCrossentropy()",
    #                                       "keras.losses.KLDivergence()"]), 
    #               metrics=["accuracy", "val_accuracy"])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr), 
                    loss=keras.losses.CategoricalCrossentropy(),
                    metrics=["accuracy", "val_accuracy"])

    return model

In [20]:
tuner = keras_tuner.BayesianOptimization(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=4,
    executions_per_trial=3,
    overwrite=True,
    directory="lstm",
    project_name="lstm",
)

In [21]:
tuner.search_space_summary()

Search space summary
Default search space size: 9
dropout_rate1 (Float)
{'default': 0.2, 'conditions': [], 'min_value': 0.2, 'max_value': 0.8, 'step': 0.025, 'sampling': 'linear'}
dropout1 (Boolean)
{'default': False, 'conditions': []}
convolution1 (Boolean)
{'default': False, 'conditions': []}
dropout2 (Boolean)
{'default': False, 'conditions': []}
convolution2 (Boolean)
{'default': False, 'conditions': []}
dropout3 (Boolean)
{'default': False, 'conditions': []}
lstm_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 1024, 'step': 32, 'sampling': 'linear'}
dropout4 (Boolean)
{'default': False, 'conditions': []}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [22]:
tuner.search(x_train[:3200], 
             y_train[:3200], 
             epochs=20, 
             validation_data=(x_val[:1500], y_val[:1500]))

Trial 2 Complete [00h 00m 02s]

Best val_accuracy So Far: None
Total elapsed time: 00h 00m 03s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
0.325             |0.4               |dropout_rate1
False             |False             |dropout1
True              |True              |convolution1
True              |False             |dropout2
False             |False             |convolution2
False             |True              |dropout3
384               |512               |lstm_units
True              |True              |dropout4
0.00047864        |0.00058563        |lr
352               |32                |filter1
2                 |2                 |kernel_size1
False             |False             |pooling1
0.3               |0.2               |dropout_rate3
0.375             |0.2               |dropout_rate4

Epoch 1/20


Traceback (most recent call last):
  File "c:\Users\agarw\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\agarw\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\agarw\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\agarw\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "c:\Users\agarw\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\agarw\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\agarw\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\agarw\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras_tuner\src\engine\tuner.py

In [ ]:
x_train.shape

(7500,)

In [ ]:
model = Sequential()
model.add(textvector_layer)
model.add(Embedding(vocab_size+1, sequence_length))
model.add(Conv1D(128, 2, activation="relu", padding="same"))
model.add(MaxPooling1D())
model.add(Conv1D(32, 2, activation="relu", padding="same"))
model.add(MaxPooling1D())
model.add(LSTM(100, activation="tanh"))
# model.add(LSTM(50, activation="tanh"))
model.add(Flatten())
model.add(Dense(150, activation="softmax"))

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.02), 
                      loss=keras.losses.CategoricalCrossentropy(),
                      metrics=["accuracy"])


epochs = 150
batch_size = 64

history = model.fit(x_train, y_train,
                    validation_data=(x_val, y_val),
                    epochs=epochs, 
                    batch_size=batch_size,
                    callbacks=[EarlyStopping(monitor='val_accuracy', 
                                             patience=15, 
                                             min_delta=0.0001)])

Epoch 1/150
 16/118 ━━━━━━━━━━━━━━━━━━━━ 2:16:41 80s/step - accuracy: 0.0072 - loss: 5.0667